In [ ]:
# GPU code to zoom into Misiurewicz point i

In [ ]:
import cupy as cp
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation

mandelbrot_kernel = cp.ElementwiseKernel(
   'float64 x, float64 y, int32 max_iter', 'int32 z',
   '''
   typedef thrust::complex<double> complex;

   complex c(x, y);
   complex value(0, 0);

   int i_iter = 0;
   for (; i_iter < max_iter && abs(value) <= 2.0; i_iter++) {
       value = value * value + c;
   }

   z = i_iter;
   ''', 'mandelbrot_kernel')

def draw_mandelbrot(centerx, centery, zoom, width, height, max_iter):
    xrange = 3.0 / zoom
    yrange = 2.5 / zoom
    x = cp.linspace(centerx - xrange/2, centerx + xrange/2, width)
    y = cp.linspace(centery - yrange/2, centery + yrange/2, height)
    X, Y = cp.meshgrid(x, y)
    Z = cp.zeros_like(X, dtype=cp.int32)
    mandelbrot_kernel(X, Y, max_iter, Z)
    return X, Y, Z

# def update(i):
#     global zoom, max_iter
#     zoom *= 1.05
#     max_iter += 0.5
#     max_iter_int = int(max_iter)
#     #centerx = -0.7 - 2e-6
#     #centery = 0.35 - 1e-14
#     centerx = 0
#     centery = 1
#     X, Y, Z = draw_mandelbrot(centerx, centery, zoom, 1024, 1024, max_iter_int)
#     im.set_array(cp.asnumpy(Z))
#     im.set_norm(colors.Normalize(vmin=0, vmax=max_iter))  # Adjust the normalization.
#     return im,

def update(i):
    global zoom, max_iter
    zoom *= 1.05
    max_iter += 0.5
    max_iter_int = int(max_iter)
    #centerx = -0.7 - 2e-6
    #centery = 0.35 - 1e-14
    centerx = 0
    centery = 1
    X, Y, Z = draw_mandelbrot(centerx, centery, zoom, 1024, 1024, max_iter_int)
    Z_np = cp.asnumpy(Z)
    im.set_array(Z_np)
    im.set_norm(colors.Normalize(vmin=Z_np.min(), vmax=Z_np.max()))  # Adjust the normalization to min and max of current image
    return im,


zoom = 1
max_iter = 400
fig, ax = plt.subplots(figsize=(7,7), dpi=150)  # Increase dpi
plt.subplots_adjust(left=0, right=1, bottom=0, top=1)  # Remove white space
ax.axis('off')  # Hide axes
norm = colors.Normalize(vmin=0, vmax=max_iter)
X, Y, Z = draw_mandelbrot(-0.7-2e-6, 0.35-1e-14, zoom, 1024, 1024, max_iter)
im = ax.imshow(cp.asnumpy(Z), origin='lower', cmap='twilight_shifted', norm=norm, extent=(cp.asnumpy(X[0,0]), cp.asnumpy(X[0,-1]), cp.asnumpy(Y[0,0]), cp.asnumpy(Y[-1,0])))

ani = FuncAnimation(fig, update, frames=range(30*23), blit=True)

ani.save('mandelbrotMisiurewicz.mp4', writer='ffmpeg', fps=30, dpi=150, bitrate=-1)  # Increase dpi and set the highest bitrate



In [ ]:
# CPU code of arbitrary precision to zoom into some point, first block uses double precision, second block uses arbitrary (SLOW)

In [ ]:
import cupy as cp
import matplotlib.pyplot as plt
import matplotlib.colors as colors

mandelbrot_kernel = cp.ElementwiseKernel(
   'float64 x, float64 y, int32 max_iter', 'int32 z',
   '''
   typedef thrust::complex<double> complex;

   complex c(x, y);
   complex value(0, 0);

   int i_iter = 0;
   for (; i_iter < max_iter && abs(value) <= 2.0; i_iter++) {
       value = value * value + c;
   }

   z = i_iter;
   ''', 'mandelbrot_kernel')

def draw_mandelbrot(centerx, centery, zoom, width, height, max_iter):
    xrange = 3.0 / zoom
    yrange = 2.5 / zoom
    x = cp.linspace(centerx - xrange/2, centerx + xrange/2, width)
    y = cp.linspace(centery - yrange/2, centery + yrange/2, height)
    X, Y = cp.meshgrid(x, y)
    Z = cp.zeros_like(X, dtype=cp.int32)
    mandelbrot_kernel(X, Y, max_iter, Z)
    return X, Y, Z

def plot_mandelbrot(centerx, centery, zoom, width=10, height=10, max_iter=256):
    d = draw_mandelbrot(centerx, centery, zoom, width, height, max_iter)
    norm = colors.Normalize(vmin=0, vmax=max_iter)
    plt.imshow(cp.asnumpy(d[2]), extent=(cp.asnumpy(d[0][0,0]), cp.asnumpy(d[0][0,-1]), cp.asnumpy(d[1][0,0]), cp.asnumpy(d[1][-1,0])),
               origin='lower', cmap='twilight_shifted', norm=norm)
    plt.show()

zoom = 1
max_iter = 256
n_steps = 120  # increase for more zoom steps

for i in range(30*23):
    zoom *= 1.05
    max_iter += 0.5
    max_iter_int = int(max_iter)

centerx = -0.7-2e-6 
centery = 0.35-1e-14
plot_mandelbrot(centerx, centery, zoom, 512, 512, max_iter_int)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from decimal import Decimal, getcontext

getcontext().prec = 50  # Set the precision you need

class ComplexDecimal:
    def __init__(self, real, imag):
        self.real = Decimal(real)
        self.imag = Decimal(imag)

    def __abs__(self):
        return (self.real**2 + self.imag**2).sqrt()

    def __add__(self, other):
        return ComplexDecimal(self.real + other.real, self.imag + other.imag)

    def __mul__(self, other):
        return ComplexDecimal(self.real * other.real - self.imag * other.imag,
                              self.real * other.imag + self.imag * other.real)

def mandelbrot(c, max_iter):
    z = ComplexDecimal(c.real, c.imag)
    for n in range(max_iter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return max_iter

def decimal_linspace(start, stop, num):
    start_dec = Decimal(start)
    stop_dec = Decimal(stop)
    step = (stop_dec - start_dec) / Decimal(num)
    return [start_dec + step * Decimal(i) for i in range(num)]

def draw_mandelbrot(centerx, centery, zoom, width, height, max_iter):
    xrange = Decimal('3.0') / Decimal(zoom)
    yrange = Decimal('2.5') / Decimal(zoom)
    x = decimal_linspace(centerx - xrange/2, centerx + xrange/2, width)
    y = decimal_linspace(centery - yrange/2, centery + yrange/2, height)
    return (x, y, np.array([[mandelbrot(ComplexDecimal(str(r), str(i)), max_iter) for r in x] for i in y]))

def plot_mandelbrot(centerx, centery, zoom, width=10, height=10, max_iter=256):
    d = draw_mandelbrot(centerx, centery, Decimal(zoom), width, height, max_iter)
    norm = colors.Normalize(vmin=0, vmax=max_iter)
    plt.imshow(d[2], origin='lower', cmap='twilight_shifted', norm=norm)  # Omit 'extent'
    plt.show()

zoom = Decimal('1')
max_iter = 256

for i in range(30*23):
    zoom *= Decimal('1.05')
    max_iter += 0.5
    max_iter_int = int(max_iter)

centerx = Decimal('-0.7')-Decimal('2e-6')
centery = Decimal('0.35')-Decimal('1e-14')
plot_mandelbrot(centerx, centery, zoom, 512, 512, max_iter_int) #512 512
